In [1]:
# imports 
import spacy

# Load the English language model
nlp = spacy.load("en_core_web_sm")

def analyze_sentence(sentence):
  """
  Analyzes a sentence and outputs ontology, NER, and dependency parsing data.

  Args:
      sentence (str): The sentence to be analyzed.

  Returns:
      dict: A dictionary containing the ontology, NER, and dependency parsing results.
  """
  doc = nlp(sentence)

  # Prepare empty results dictionary
  results = {
      "ontology": [],
      "ner": [],
      "dependency_parsing": []
  }

  # Process each token
  for token in doc:
    # Extract basic information
    text = token.text
    pos = token.pos_
    dep_ = token.dep_
    head = token.head.text

    # Named Entity Recognition (NER)
    entity_text = ""
    entity_type = ""
    if token.ent_type_:
      entity_text = text
      entity_type = token.ent_type_
    results["ner"].append({"text": entity_text, "type": entity_type})

    # Ontology Table (basic implementation using POS)
    if pos in ["NOUN", "PROPN"]:  # Check for nouns and proper nouns (potential entities)
      relation = ""  # Placeholder for relation (more complex logic needed)
      attributes = []  # Placeholder for attributes (more complex logic needed)
      results["ontology"].append({
          "text": text,
          "type": "Entity" if entity_type else "Potential Entity",
          "relation": relation,
          "attributes": attributes
      })

    # Dependency Parsing
    results["dependency_parsing"].append({
        "word": text,
        "pos": pos,
        "dependency": dep_,
        "head": head
    })

  return results

# Example usage
sentence = "The CEO of Apple, Tim Cook, announced a new product launch."
analysis_result = analyze_sentence(sentence)

# Print the results (further processing might be needed)
print("Ontology Table:")
print(analysis_result["ontology"])
print("\nNER Table:")
print(analysis_result["ner"])
print("\nDependency Parsing Table:")
print(analysis_result["dependency_parsing"])



c:\Users\Ahmed El Agamy\AppData\Local\Programs\Python\Python310\lib\site-packages\thinc\compat.py:36: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  hasattr(torch, "has_mps")
c:\Users\Ahmed El Agamy\AppData\Local\Programs\Python\Python310\lib\site-packages\thinc\compat.py:37: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  and torch.has_mps  # type: ignore[attr-defined]
c:\Users\Ahmed El Agamy\AppData\Local\Programs\Python\Python310\lib\site-packages\spacy\util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.5.0) was trained with spaCy v3.5.0 and may not be 100% compatible with the current version (3.7.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


Ontology Table:
[{'text': 'CEO', 'type': 'Potential Entity', 'relation': '', 'attributes': []}, {'text': 'Apple', 'type': 'Entity', 'relation': '', 'attributes': []}, {'text': 'Tim', 'type': 'Entity', 'relation': '', 'attributes': []}, {'text': 'Cook', 'type': 'Entity', 'relation': '', 'attributes': []}, {'text': 'product', 'type': 'Potential Entity', 'relation': '', 'attributes': []}, {'text': 'launch', 'type': 'Potential Entity', 'relation': '', 'attributes': []}]

NER Table:
[{'text': '', 'type': ''}, {'text': '', 'type': ''}, {'text': '', 'type': ''}, {'text': 'Apple', 'type': 'ORG'}, {'text': '', 'type': ''}, {'text': 'Tim', 'type': 'PERSON'}, {'text': 'Cook', 'type': 'PERSON'}, {'text': '', 'type': ''}, {'text': '', 'type': ''}, {'text': '', 'type': ''}, {'text': '', 'type': ''}, {'text': '', 'type': ''}, {'text': '', 'type': ''}, {'text': '', 'type': ''}]

Dependency Parsing Table:
[{'word': 'The', 'pos': 'DET', 'dependency': 'det', 'head': 'CEO'}, {'word': 'CEO', 'pos': 'NOUN',

In [2]:
import spacy
from nltk.corpus import wordnet as wn


def analyze_sentence(sentence):
  """
  Analyzes a sentence and outputs ontology, NER, and dependency parsing data with 
  improvements for relation/attribute identification, lexical resources, coreference 
  resolution, and disambiguation.

  Args:
      sentence (str): The sentence to be analyzed.

  Returns:
      dict: A dictionary containing the ontology, NER, and dependency parsing results.
  """
  doc = nlp(sentence)

  # Prepare empty results dictionary
  results = {
      "ontology": [],
      "ner": [],
      "dependency_parsing": []
  }

  # Coreference resolution (using simple heuristic)
  coref_chains = {}
  for token in doc:
    if token.dep_ == "nsubj":  # Identify potential subjects
      entity_text = token.text
      if entity_text in coref_chains:
        coref_chains[entity_text].append(token.idx)
      else:
        coref_chains[entity_text] = [token.idx]

  # Process each token
  for token in doc:
    text = token.text
    pos = token.pos_
    dep_ = token.dep_
    head = token.head.text

    # Named Entity Recognition (NER)
    entity_text = ""
    entity_type = ""
    if token.ent_type_:
      entity_text = text
      entity_type = token.ent_type_
    results["ner"].append({"text": entity_text, "type": entity_type})

    # Ontology Table with relation and attribute logic
    if pos in ["NOUN", "PROPN"]:
      relation = ""
      attributes = []

      # Check for verb phrase as relation (consider dependency parsing)
      if dep_ == "ROOT" or dep_ == "verb":
        relation = head

      # Identify potential attributes using part-of-speech and dependency parsing
      for child in token.children:
        if child.pos_ in ["ADJ", "ADP"]:  # Adjectives or adpositional phrases
          attributes.append(child.text)

      # Coreference resolution (apply to previously identified entity)
      if token.idx in coref_chains:
        entity_text = coref_chains[entity_text][0]  # Use first mention as reference
        entity_text = doc[entity_text].text

      results["ontology"].append({
          "text": entity_text,
          "type": "Entity" if entity_type else "Potential Entity",
          "relation": relation,
          "attributes": attributes
      })

    # Dependency Parsing
    results["dependency_parsing"].append({
        "word": text,
        "pos": pos,
        "dependency": dep_,
        "head": head
    })

  # WordNet integration (enrich ontology with hypernyms/synonyms)
  for entity in results["ontology"]:
    if entity["text"]:
      synsets = wn.synsets(entity["text"])
      if synsets:
        entity["hypernyms"] = [synset.hypernyms() for synset in synsets][0]  # Get first hypernym
        entity["synonyms"] = [lemma.name() for synset in synsets for lemma in synset.lemmas()]

  # Disambiguation (simple example using WordNet glosses)
  for entity in results["ontology"]:
    if entity["relation"] and entity["text"]:
      synsets = wn.synsets(entity["text"])
      if synsets:
        gloss_matches = [gloss for synset in synsets for gloss in synset.definition().split() if entity["relation"] in gloss]
        if gloss_matches:
          entity["relation"] = gloss_matches[0]  # Choose first gloss match (can be improved)

  return results

# Example usage
sentence = "The CEO of Apple, Tim Cook, announced a new product launch."
analysis_result = analyze_sentence(sentence)

# Print the results (further processing might be needed)
print("Ontology Table:")
print(analysis_result["ontology"])
print("\nNER Table:")
print(analysis_result["ner"])
print("\nDependency Parsing Table:")
print(analysis_result["dependency_parsing"])


Ontology Table:
[{'text': '', 'type': 'Potential Entity', 'relation': '', 'attributes': ['of']}, {'text': 'Apple', 'type': 'Entity', 'relation': '', 'attributes': [], 'hypernyms': [Synset('edible_fruit.n.01'), Synset('pome.n.01')], 'synonyms': ['apple', 'apple', 'orchard_apple_tree', 'Malus_pumila']}, {'text': 'Tim', 'type': 'Entity', 'relation': '', 'attributes': []}, {'text': 'Cook', 'type': 'Entity', 'relation': '', 'attributes': [], 'hypernyms': [Synset('skilled_worker.n.01')], 'synonyms': ['cook', 'Cook', 'James_Cook', 'Captain_Cook', 'Captain_James_Cook', 'cook', 'cook', 'fix', 'ready', 'make', 'prepare', 'cook', 'fudge', 'manipulate', 'fake', 'falsify', 'cook', 'wangle', 'misrepresent', 'cook']}, {'text': '', 'type': 'Potential Entity', 'relation': '', 'attributes': []}, {'text': '', 'type': 'Potential Entity', 'relation': '', 'attributes': ['new']}]

NER Table:
[{'text': '', 'type': ''}, {'text': '', 'type': ''}, {'text': '', 'type': ''}, {'text': 'Apple', 'type': 'ORG'}, {'tex

In [3]:
analysis_result["ontology"]

[{'text': '',
  'type': 'Potential Entity',
  'relation': '',
  'attributes': ['of']},
 {'text': 'Apple',
  'type': 'Entity',
  'relation': '',
  'attributes': [],
  'hypernyms': [Synset('edible_fruit.n.01'), Synset('pome.n.01')],
  'synonyms': ['apple', 'apple', 'orchard_apple_tree', 'Malus_pumila']},
 {'text': 'Tim', 'type': 'Entity', 'relation': '', 'attributes': []},
 {'text': 'Cook',
  'type': 'Entity',
  'relation': '',
  'attributes': [],
  'hypernyms': [Synset('skilled_worker.n.01')],
  'synonyms': ['cook',
   'Cook',
   'James_Cook',
   'Captain_Cook',
   'Captain_James_Cook',
   'cook',
   'cook',
   'fix',
   'ready',
   'make',
   'prepare',
   'cook',
   'fudge',
   'manipulate',
   'fake',
   'falsify',
   'cook',
   'wangle',
   'misrepresent',
   'cook']},
 {'text': '', 'type': 'Potential Entity', 'relation': '', 'attributes': []},
 {'text': '',
  'type': 'Potential Entity',
  'relation': '',
  'attributes': ['new']}]